# Import a Molecules/Atom from cartesian coordinates:
**Task:** 
1. Create a Molecule object from its cartesian coordinates
2. Ask for its atomic and molecular properties

## [Water molecule](https://en.wikipedia.org/wiki/Properties_of_water) ($H_{2}O$):

<img 
src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/H2O_2D_labelled.svg/2560px-H2O_2D_labelled.svg.png"
alt="water" 
width="200" 
height="100" 
/>

### Properties:
- Number of molecules: 1
- Number of atoms: 3
- Atomic symbols: O, H, H
- Atomic masses [g/mol]: 
    - O: 15.999
    - H: 1.008
    - H: 1.008
- Molar mass [g/mol]: 18.015
- Cartesian coordinates [Angstrom]:
    ```XYZ
    O    0.00000    0.00000    0.00000
    H    0.58708    0.75754    0.00000
    H    0.58708   -0.75754    0.00000
    ```
- Bond distance [Angstrom]:
    - H-O: 0.9584 (95.84 pm)
    - H-H: 1.5151
- Bond angle [Degrees]:
    - H-O-H: 104.45
    - H-H-O: 37.78

In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'src' directory is out ('..') of our current directory 

import os
import sys
module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from src.base_molecule import Molecule
# Creates a molecule. You can use a list, a dictionary (the key MUST be "atoms")
# or another molecule object (see below)

# 1 Option
water=[("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]
water_molecule = Molecule(water)

# 2 Option
water_dict = {"atoms": [("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)]}
water_molecule = Molecule(water_dict)



In [3]:
## See initial molecule.

import py3Dmol

water_molecule = Molecule(water_dict)
water_molecule = water_molecule.translate(0, x=1.5, y=1.5, z=1.5)
bi_water = water_molecule.add_fragments(water_dict)


xyz_view = py3Dmol.view(width=300,height=200)
xyz_view.addModel(str(bi_water),'xyz')
xyz_view.setStyle({'stick':{}})

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
from copy import deepcopy
from numpy import random
import py3Dmol

from src.base_molecule import Molecule

def random_move(initial_cluster : Molecule, fragment = 0):
    # Random step of every fragment, staying inside sphere
    random_gen = random.default_rng()

    #Approximately 3 bohr radius for each atom
    sphere_radius = initial_cluster.total_atoms*1.5*0.5


    #All fragments will be moved AND rotated
    final_cluster_serie_xyz = ""


    final_cluster = deepcopy(initial_cluster)
    for j in range(0,final_cluster.total_fragments):
        # angle between [0, 360)
        ax = random_gen.uniform() * 360
        ay = random_gen.uniform() * 360
        az = random_gen.uniform() * 360

        # moving between [-move, +move]
        move = sphere_radius / 1.6
        tx = move * (random_gen.uniform() - 0.5 )
        ty = move * (random_gen.uniform() - 0.5 )
        tz = move * (random_gen.uniform() - 0.5 )

        final_cluster = final_cluster.translate(0, x=tx, y=ty, z=tz)
        final_cluster = final_cluster.rotate(final_cluster.total_fragments-1, x=ax, y=ay, z=az)

    # saving coordinates as a string
    final_cluster_serie_xyz += final_cluster.xyz

    #Checking if any fragments falls out of the sphere (ALREADY CHECKED)
    import math as m
    if m.sqrt(final_cluster.coordinates[1][1]*final_cluster.coordinates[1][1]+final_cluster.coordinates[1][2]*final_cluster.coordinates[1][2]+final_cluster.coordinates[1][3]*final_cluster.coordinates[1][3]) > sphere_radius:
        print("un fragmento está cayendo fuera de la esfera")
    return final_cluster

#Visualization
def visualization_random_process(serie_string : str, n_atoms):
    #Approximately 3 bohr radius for each atom
    sphere_radius = n_atoms*1.5*0.5
    xyz_view = py3Dmol.view(width=1000,height=700)
    xyz_view.addModelsAsFrames(serie_string,'xyz')
    xyz_view.setStyle({'sphere': {'radius': 0.8}})
    xyz_view.addSphere({'center': {'x':0, 'y':0, 'z':0}, 
        'radius': sphere_radius , 
        'color' :'yellow',
        'alpha': 0.5,
        })

    xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 10})
    xyz_view.zoomTo()
    xyz_view.show()

#### Testing random moves inside sphere
total_steps = 10

cluster_of_interest = Molecule(
    water,
    water,
    water,
    water
)

#All fragments will be moved AND rotated
serie_of_interest_xyz = ""

for i in range(total_steps):
    serie_of_interest_xyz += random_move(cluster_of_interest).xyz

visualization_random_process(serie_of_interest_xyz,cluster_of_interest.total_atoms)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [28]:
module_path = os.path.abspath(os.path.join('src'))
sys.path.append(module_path)
print(module_path)
from src.search_configuration import SearchConfig

#################################################
# Configuración del sistema de trabajo
#################################################
water1 = {
    "atoms": [
        ("H", 1.4891244, 0., 1.0332262),
        ("O", 0., 0., -0.1302053),
        ("H", -1.4891244, 0., 1.0332262),
    ]
}
water2 = {
    "atoms": [
        ("H", 2.4891244, 1., 2.0332262),
        ("O", 1., 1., 0.8697947),
        ("H", -0.4891244, 1., 2.0332262),
    ],
}

h2o = Molecule(water1, water2)

#Configuración de busqueda
#a = 0.5
#"bounds": [(-a, a),(-a, a),(-a, a),(-a, a),(-a, a),(-a, a)],
################################################
# Configuración de la busqueda
###############################################
search_setting  = {
    "basis": 'sto-3g',
    "search_methodology": 2, #1 dual_annealing 2 shgo
    "type of fragments": 0,
    "program_cost_function": 1,
    }

search_config = SearchConfig(h2o, search_setting)

search_config.run()
print(search_config._search)

/home/danian/famaf/ascec/workflow/testing/src
*** Calculo Realizado con SHGO from Scipy ***
converged SCF energy = -149.267832506923
converged SCF energy = -149.267832506924
converged SCF energy = -148.919016458359
converged SCF energy = -148.949841885541
converged SCF energy = -148.919107328621
converged SCF energy = -148.908023381707


KeyboardInterrupt: 

In [30]:
#Visualization
def visualization(serie_string : str, n_atoms):
    #Approximately 3 bohr radius for each atom
    sphere_radius = n_atoms*1.5*0.5
    xyz_view = py3Dmol.view(width=1000,height=700)
    xyz_view.addModelsAsFrames(serie_string,'xyz')
    xyz_view.setStyle({'sphere': {'radius': 0.8}})
    xyz_view.addSphere({'center': {'x':0, 'y':0, 'z':0}, 
        'radius': sphere_radius , 
        'color' :'yellow',
        'alpha': 0.5,
        })

    xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 3})
    xyz_view.zoomTo()
    xyz_view.show()

#El archivo configurations.xyz es guardado en la dirección de este jupyter.notebook
with open("configurations.xyz","r") as outxyz:
    serie_string = outxyz.read()
    visualization(serie_string,h2o._total_atoms)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# Atomic Simulation Environment
# https://wiki.fysik.dtu.dk/ase/index.html
# !pip install --upgrade --user ase


# ChemML
# https://hachmannlab.github.io/chemml/index.html
# !pip install chemml

In [ ]:
# NGLview
# https://github.com/nglviewer/nglview
# !pip install nglview

# ---------------------------------------
# pytraj 
# https://amber-md.github.io/pytraj/latest/index.html
# !pip install pytraj
